In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def firstFit(result, item):
    #print('firstFit')
    allocated = False
    for i, bin in enumerate(result):
        if bin + item <= capasity:
            result[i] += item
            allocated = True
            break
        
    if not allocated:
        result.append(item)
        
    return result

In [3]:
def nextFit(result, item):
    #print('nextFit')
    allocated = False
    if len(result) > 0:
        bin = result[len(result) - 1]
        if bin + item <= capasity:
            result[len(result) - 1] += item
            allocated = True
        
    if not allocated:
        result.append(item)
        
    return result

In [4]:
def bestFit(result, item):
    #print('bestFit')
    best = float('inf')
    pos = None
    for i, bin in enumerate(result):
        if bin + item <= capasity:
            space = capasity - (bin + item)
            if space < best:
                best = space
                pos = i
                break
        
    if pos == None:
        result.append(item)
    else:
        result[pos] += item
        
    return result

In [5]:
def worstFit(result, item):
    #print('worstFit')
    worst = -1
    pos = None
    for i, bin in enumerate(result):
        if bin + item <= capasity:
            space = capasity - (bin + item)
            if space > worst:
                worst = space
                pos = i
                break
        
    if pos == None:
        result.append(item)
    else:
        result[pos] += item
        
    return result

In [6]:
def fitness(result):
    return np.sum(np.square(np.array(result) / capasity)) / len(result) 

In [7]:
def applyChromosome(items, chromosome):
    result = []
    idx = 0
    while idx < len(items):
        for heuristic in chromosome:
            if idx < len(items):
                heuristic(result, items[idx])
                idx += 1
                #print(result)
            else:
                break
            
    return result

In [8]:
def generateChromosome():
    chromosome = []
    chromosomeLength = np.random.randint(low=MIN_CHROMOSOME_SIZE,high=MAX_CHROMOSOME_SIZE)
    for i in range(chromosomeLength):
        heuristicChoice = np.random.randint(len(heuristicList))
        chromosome.append(heuristicList[heuristicChoice])
        
    return chromosome

In [9]:
def createPopulation():
    population = []
    for i in range(POPULATION_SIZE):
        population.append(generateChromosome())
        
    return population

In [10]:
def getPopulationFitness(population):
    populationResults = []
    for individual in population:
        populationResults.append(applyChromosome(items, individual))
        
    populationFitness = []
    for result in populationResults:
        populationFitness.append([fitness(result), len(result)])
        
    return populationFitness, populationResults

In [11]:
def swap(chromosome, times=1):
    clone = chromosome[:]
    
    for _ in range(times):
        pos1 = np.random.randint(len(clone))
        pos2 = np.random.randint(len(clone))
        tmp = clone[pos1]
        clone[pos1] = clone[pos2]
        clone[pos2] = tmp
        
    return clone

In [12]:
def tournamentSelector(population, tournament_size=5):
    # Make the tournament
    random_indicies = np.random.randint(len(population), size=tournament_size).tolist()
    tournament = []
    for idx, val in np.ndenumerate(random_indicies):
        tournament.append(population[val])

    # Run the tournament
    fitnesss, results = getPopulationFitness(tournament)
    
    maxPos = np.argmax(fitnesss, axis=0)[0]
    minPos = np.argmin(fitnesss, axis=0)[0]
    
    return [random_indicies[maxPos], fitnesss[maxPos]], [random_indicies[minPos], fitnesss[minPos]]

In [13]:
location = '../experiments/problems/'

problems = [
            ['bin1data/N1C1W1_A.BPP', 25],
            ['bin1data/N1C1W1_O.BPP', 32],
            ['bin1data/N1C1W1_P.BPP', 26],
            ['bin1data/N1C1W1_T.BPP', 28],
            ['bin1data/N1C1W1_G.BPP', 25],
            ['bin1data/N1C1W1_K.BPP', 26],
            ['bin1data/N1C1W1_S.BPP', 28],
            ['bin1data/N1C1W1_F.BPP', 27],
    
            ['bin2data/N1W1B1R0.BPP', 18],
            ['bin2data/N1W1B1R1.BPP', 18],
            ['bin2data/N1W2B3R2.BPP', 10]
        ]

heuristicList = [
    firstFit,
    nextFit,
    bestFit,
    worstFit
]

problemIndex = 2
problemSet = pd.read_csv(location + problems[problemIndex][0], header=None).values.tolist()

PROBLEM_SIZE = problemSet.pop(0)[0]
BIN_CAPACITY = problemSet.pop(0)[0]

items = pd.DataFrame(problemSet)
items = np.array(items[0])

# items = [5, 3, 7, 2, 9, 4, 8, 9, 3, 5, 5, 3, 7, 2, 9, 4, 8, 9, 3, 5]
assert PROBLEM_SIZE == len(items)

capasity = BIN_CAPACITY

MIN_CHROMOSOME_SIZE = 5
MAX_CHROMOSOME_SIZE = PROBLEM_SIZE

POPULATION_SIZE = 5
GENERATIONS = 100

population = createPopulation()
#population = [[heuristicList[0]], [heuristicList[1]], [heuristicList[2]], [heuristicList[3]]]

endResult = []

for gen in range(GENERATIONS):    
    best, worst = tournamentSelector(population, tournament_size=6)
    if gen % 10 == 0:
        fitnesss, results = getPopulationFitness(population)
        max = np.argmax(fitnesss, axis=0)
        endResult.append(results[max[0]])
        print(fitnesss[max[0]], results[max[0]])
        if problems[problemIndex][1] == fitnesss[max[0]][1]:
            print('=== Found known min: ', problems[problemIndex][1], '===')
            break
    
    population[worst[0]] = swap(population[best[0]], 1)


[0.8172814814814815, 27] [100, 94, 100, 97, 99, 100, 97, 88, 100, 99, 100, 100, 100, 100, 99, 99, 99, 93, 88, 87, 59, 58, 57, 55, 98, 97, 13]
[0.8532692307692307, 26] [96, 94, 100, 97, 99, 100, 97, 88, 100, 99, 100, 100, 100, 100, 99, 99, 99, 95, 90, 88, 86, 58, 57, 55, 98, 82]
=== Found known min:  26 ===


In [14]:
assert np.sum(items) == np.sum(endResult[len(endResult) - 1])